In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as miss

## Preprocessing tools
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

## Models & evaluation metrics
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import joblib

## setting random state for reproducibility
SEED = 321
np.random.seed(SEED)
## Matplotlib style
fav_style = ('ggplot','tableau-colorblind10')
fav_context  ={'context':'notebook', 'font_scale':1.1}
plt.style.use(fav_style)
sns.set_context(**fav_context)
plt.rcParams['savefig.transparent'] = False
plt.rcParams['savefig.bbox'] = 'tight'

In [2]:
import joblib
loaded = joblib.load('reg_l01.joblib')
loaded.keys()


dict_keys(['X_train', 'y_train', 'X_test', 'y_test', 'preprocessor', 'Linear Regression'])

In [3]:
X_train_df = loaded['X_train']
y_train = loaded['y_train']
X_test_df = loaded['X_test']
y_test = loaded['y_test']
preprocessor = loaded['preprocessor']
loaded_model = loaded['Linear Regression']

In [4]:
## Fitting and evaluating the model
rf_reg = LinearRegression()
rf_reg.fit(X_train_df, y_train)
evaluate_regression(rf_reg, X_train_df, y_train, X_test_df, y_test)

ValueError: could not convert string to float: 'NCG06'

In [ ]:
evaluate_regression(loaded_model,X_train_df, y_train, X_test_df,y_test)
loaded_importances = get_importances(loaded_model)
plot_importance_color(loaded_importances,top_n=15,color_dict=None);

# Model 1 Linear Regression

In [ ]:
def evaluate_regression(model, X_train,y_train, X_test, y_test,for_slides=True): 
    """Evaluates a scikit learn regression model using r-squared and RMSE
    FOR SLIDES VERS DOES MULTIPLE PRINT STATEMENTS FOR VERTICAL DISPLAY OF INFO"""
    
    ## Training Data
    y_pred_train = model.predict(X_train)
    r2_train = metrics.r2_score(y_train, y_pred_train)
    rmse_train = metrics.mean_squared_error(y_train, y_pred_train, 
                                            squared=False)
    mae_train = metrics.mean_absolute_error(y_train, y_pred_train)
    

    ## Test Data
    y_pred_test = model.predict(X_test)
    r2_test = metrics.r2_score(y_test, y_pred_test)
    rmse_test = metrics.mean_squared_error(y_test, y_pred_test, 
                                            squared=False)
    mae_test = metrics.mean_absolute_error(y_test, y_pred_test)
    
    if for_slides:
        df_version =[['Split','R^2','MAE','RMSE']]
        df_version.append(['Train',r2_train, mae_train, rmse_train])
        df_version.append(['Test',r2_test, mae_test, rmse_test])
        df_results = pd.DataFrame(df_version[1:], columns=df_version[0])
        df_results = df_results.round(2)
        display(df_results.style.hide(axis='index').format(precision=2, thousands=','))
        
    else: 
        print(f"Training Data:\tR^2 = {r2_train:,.2f}\tRMSE = {rmse_train:,.2f}\tMAE = {mae_train:,.2f}")
        print(f"Test Data:\tR^2 = {r2_test:,.2f}\tRMSE = {rmse_test:,.2f}\tMAE = {mae_test:,.2f}")

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg= LinearRegression()
lin_reg.fit(X_train_df,y_train)
evaluate_regression(lin_reg, X_train_df, y_train, 
                    X_test_df, y_test)

In [ ]:
de.plot_residuals(lin_reg, X_test_df,y_test)

# Extracting and Visualizing Coefficients

In [ ]:
def get_coefficients(lin_reg):
    coeffs = pd.Series(lin_reg.coef_, index= lin_reg.feature_names_in_)
    coeffs.loc['intercept'] = lin_reg.intercept_
    return coeffs

In [ ]:
coeffs = get_coefficients(lin_reg)
coeffs

In [ ]:
pd.set_option('display.float_format',lambda x: f"{x:,.4f}")
pd.set_option('display.max_rows', None)

In [ ]:
coeffs.sort_values()

In [ ]:
country_feats = [c for c in X_train_df.columns if c.startswith('Country')]
len(country_feats)

In [ ]:
ax = coeffs[country_feats].sort_values().plot(kind='barh', figsize=(8,26))
ax.axvline(0, color='k')
ax.set(xlabel='Life Expectancy', title="Coefficients - Countries");

# Model 2 Decision Tree Regressor

In [ ]:
dec_tree = DecisionTreeRegressor(random_state = 42)

tree_preprocessor = make_column_transformer(number_tuple, category_tuple, remainder = "passthrough")

dec_tree_pipe = make_pipeline(preprocessor, dec_tree)

dec_tree_pipe.fit(X_train, y_train)

print(dec_tree.get_depth())

test_preds = dec_tree_pipe.predict(X_test)

test_score = dec_tree_pipe.score(X_test, y_test)
print(test_score)

In [ ]:
depth = dec_tree.get_depth()
print(depth)

In [ ]:
dec_tree.get_params()

In [ ]:
dec_tree.get_n_leaves()

In [ ]:
depths = list(range(2, depth + 1)) 
scores = pd.DataFrame(index=depths, columns=['Test Score','Train Score'])
for depth in depths:
    dec_tree = DecisionTreeRegressor(max_depth=depth, random_state=42)
    dec_tree_pipe = make_pipeline(tree_preprocessor, dec_tree)
    dec_tree_pipe.fit(X_train, y_train)
    train_score = dec_tree_pipe.score(X_train, y_train)
    test_score = dec_tree_pipe.score(X_test, y_test)
    scores.loc[depth, 'Train Score'] = train_score
    scores.loc[depth, 'Test Score'] = test_score
sorted_scores = scores.sort_values(by='Test Score', ascending=False)
sorted_scores.head();

In [ ]:
dec_tree_opt = DecisionTreeRegressor(max_depth = 11, random_state = 42)
dec_tree_pipe = make_pipeline(tree_preprocessor, dec_tree_opt)
dec_tree_pipe.fit(X_train, y_train)
test_score = dec_tree_pipe.score(X_test, y_test);

In [ ]:
test_MAE = mean_absolute_error(y_test, dec_tree_pipe.predict(X_test))
test_MSE = mean_squared_error(y_test, dec_tree_pipe.predict(X_test))
test_RMSE = np.sqrt(test_MSE)
print(test_score) #This is r^2
print(test_MAE)
print(test_MSE)
print(test_RMSE)